# 修了課題　- 深層学習：前編２

### AutoRaceのWebクローラー構築

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import urllib
import time
from datetime import datetime, timedelta
from pathlib import Path

def get_data_url(data_nums, rno):
    base = "http://autorace.jp/netstadium/RaceResult"
    baseAdd1 = "/sanyou/"
    url = base + baseAdd1 + data_nums + "_" + rno
    return urllib.parse.quote_plus(url, "/:?=&")

def date_span(start_date, end_date):
    """start_date、end_dateの期間に含まれる日毎のdatetimeオブジェクトを返すジェネレータ
    """
    for i in range((end_date - start_date).days + 1):
        yield start_date + timedelta(i)

        
start_date = datetime.strptime('20170823', '%Y%m%d')
end_date = datetime.strptime('20170823', '%Y%m%d')

today = datetime.today()
date_prefix = today.strftime('%Y-%m-%d')
file_origin = 'crawl_' + date_prefix + '_Crawler.csv'

my_file = Path(file_origin)
isFirst = True

if my_file.is_file():
    mainDf = pd.read_csv(file_origin, index_col=0, header=0)
    isFirst = False
else:
    mainDf = None

race_noms = []
for i in range(1, 13):
    conv_num = str(i).zfill(1)
    race_noms.append(conv_num) 

sleep_time = 5

for target_date in date_span(start_date, end_date):
    for rno in race_noms:
        target_url = get_data_url(target_date.strftime('%Y-%m-%d'), rno)
        print(target_url)
        headers = {'User-Agent': 'Mozilla/5.0'}
        time.sleep(sleep_time)
        response = requests.get(target_url, headers=headers)# <Response [200]>
        soup = BeautifulSoup(response.text, 'html.parser')
        content3 = soup.find_all("table", class_="tblMain tblBasic mg_btm_15")
#        print(content3)
        for content3_part in content3:
            jyunni = [td.text for td in content3_part.find_all("td", class_="light f16")]
            name = [td.text for td in content3_part.find_all("td", class_="f8")]
            name = [row[:].replace('\n', '').replace('\t', '') for row in name]
            raceDict = pd.DataFrame({"jyunni": jyunni, "race_number": rno, "name": name})
            print(raceDict)
            if mainDf is None:
                raceDict.to_csv(file_origin)
                mainDf = raceDict
                print("新規作成: " + file_origin)
            else:
                raceDict.to_csv(file_origin, mode='a', header=False)
                mainDf = mainDf.append(raceDict)
                print("データ追記: " + file_origin)


http://autorace.jp/netstadium/RaceResult/sanyou/2017-08-23_1
  jyunni race_number                           name
0      1           1         OKAMATSU TADASHI      
1      2           1        TAKENAKA KAZUNARI      
2      3           1          MASUZAKI YOSUKE      
3      4           1          KANAYAMA SYUHEI      
4      5           1           HAYAFUNE AYUMU      
5      6           1           MATSUBU SHINJI      
6      7           1          NAKANO MASANORI      
7      8           1           ASANO HIROYUKI      
新規作成: crawl_2020-01-03_Crawler.csv
http://autorace.jp/netstadium/RaceResult/sanyou/2017-08-23_2
  jyunni race_number                           name
0      1           2            GOSYO ATSUSHI      
1      2           2          SHIGETOMI HIDEO      
2      3           2           IKEDA MASAKAZU      
3      4           2         ARIYOSHI TATSUYA      
4      5           2        FUJIMOTO TSUYOSHI      
5      6           2        NISHIKAWA YORIOMI      
6      7   